In [ ]:
using BenchmarkTools
using ProfileView

# Julia Likes to Know Your Type

### Suppose I want to make a container that will hold two arrays, then I want to make a function that operates on those two arrays

In [ ]:
# First define a basic container
struct Holder
    x
    y
end

In [ ]:
# And now my function
a = 7
b = 3
function f(H::Holder)
    return a * H.x.^2 + b * H.y.^2
end;


In [ ]:
# Now make a container and call f
H = Holder([1, 2], [2, 3])
f(H)

In [ ]:
# Looks good, except...
@time f(H)

In [ ]:
# That a lot of memory allocations for a simple function
# and if the containers get big...
H = Holder(rand(10^7), rand(10^7))
f(H)
@time f(H);

In [ ]:
# Let's do better
# First thing we can do is look at what the compiler sees
H = Holder([1, 2], [2, 3])
@code_warntype f(H);

In [ ]:
# We should never use global variables inside a function, because Julia doesn't know if I'll change the type
a = "This was an Int but now it's a String..."
f(H)

In [ ]:
# So we can make variables constants or we can make them function arguments
const ca = 7
function f(H, b)
    return ca * H.x.^2 + b * H.y.^2
end

H = Holder([1, 2], [2, 3])
f(H, 3) # compile
@time f(H, 3);
println()
@code_warntype f(H, 3);

# this didn't actually reduce allocations in this case, but the compiler looks better
# it's a start

In [ ]:
# So it knows what ca and b are, but not what H1.x and H1.y are
Holder("Howdy", (1.0, [1+2im, 1.0+3im]))

#Let's be more specific

In [ ]:
# This container provides some type information
struct TypedHolder
    x::AbstractVector{<:Real}
    y::AbstractVector{<:Real}
end
TH = TypedHolder([1, 2], [2, 3])

In [ ]:
f(TH, 3)
@time f(TH, 3)
println()
@code_warntype f(TH, 3)

In [ ]:
# The problem is that we've still used abstract types, so maybe it's a bit better, but the compiler still doesn't know what it's getting
# We should use concrete types instead
struct ConcreteHolder
    x::Vector{Int}
    y::Vector{Int}
end
CH = ConcreteHolder([1, 2], [2, 3])

In [ ]:
f(CH, 3)
@time f(CH, 3)
println()
@code_warntype f(CH, 3)

In [ ]:
# Much better! But now we're pretty restricted
CH = ConcreteHolder(rand(10^7), rand(10^7));

### The Power of Parametric Types

In [ ]:
struct ParametricElementHolder{T<:Real}
    x::Vector{T}
    y::Vector{T}
end
ParametricElementHolder([1, 2], [2, 3])
ParametricElementHolder(rand(10^7), rand(10^7));

In [ ]:
PEH = ParametricElementHolder([1, 2], [2, 3])

f(PEH, 3)
@time f(PEH, 3)
println()
@code_warntype f(PEH, 3)


In [ ]:
# This is still a bit restrictive
println(typeof(1:2))
ParametricElementHolder(1:2, 2:3)

In [ ]:
# So we can be more general
struct ParametricHolder{T<:AbstractVector{<:Real}, S<:AbstractVector{<:Real}}
    x::T
    y::S
end

# Now all of these work
println(typeof(ParametricHolder([1, 2], [2, 3])))
println(typeof(ParametricHolder(rand(10^7), rand(10^7))))
println(typeof(ParametricHolder(1:2, 2:3)))
println(typeof(ParametricHolder(1:2, [2.0, 3.0])))

# And we've got better performance
PH = ParametricHolder([1, 2], [2, 3])

f(PH, 3)
@time f(PH, 3)
println()
@code_warntype f(PH, 3)

## It's all about the dots

In [ ]:
# Here's the function we've been running
function f(C, b)
    return ca * C.x .^ 2 + b * C.y .^ 2
end

# Those dots say to operate element wise, but most of the operators don't have dots.
# This is the same as
function f2(C, b)
    x2 = C.x .^ 2
    ax2 = ca * x2
    y2 = C.y .^ 2
    by2 = b * y2
    return ax2 + by2
end

f(PH, 3)
f2(PH, 3)
@time f(PH, 3)
@time f2(PH, 3);

In [ ]:
# Instead we should use dots so that all of it is vectorized
function fdot(C, b)
    return ca .* C.x .^ 2 .+ b .* C.y .^ 2
end
fdot(PH, 3)
@time fdot(PH, 3);

In [ ]:
# There's still some allocation because we're allocating the result
# We could preallocate instead

function fdot!(result, C, b)
    result .= ca .* C.x .^ 2 .+ b .* C.y .^ 2
end

R = zero(PH.x)
fdot!(R, PH, b)
@time fdot!(R, PH, b);

In [ ]:
# Success! We've eliminated all allocations
# Let's see performance
x = rand(10^7)
y = rand(10^7)

H = Holder(x, y)
PH = ParametricHolder(x, y)

R = zero(x)

f(H, 3)
f(PH, 3)
fdot!(R, H, 3)
fdot!(R, PH, 3)

println("No Parametric Type, No dots")
@time f(H, 3)
println("Parametric Type, No dots")
@time f(PH, 3)
println("No Parametric Type, with dots")
@time fdot!(R, H, 3)
println("Parametric Type, with dots")
@time fdot!(R, PH, 3);


### We can see that the dots give the main performance increase, but to fully eliminate allocations, we needed to use parametric types. In this simple application, the parametric type didn't increase performance, but if this were inside a loop, an optimizer, or any performance critical part of the code, it can be a huge benefit.

# Avoid allocation whenever possible in loops/optimization

In [ ]:
function xpm(x)
    return [x-1, x, x+1]
end;

function g()
    y = 0
    for i = 1:10^7
        ret = xpm(i)
        y += ret[2]
    end
    return y
end;
g()
@time g();
# xpm allocates every time, so this is bad

In [ ]:
# Instead we could preallocate and then operate in-place
function xpm!(ret, x)
    return [x-1, x, x+1]
end

function g_inplace()
    y = 0
    ret = Vector{Int}(undef, 3)
    for i = 1:10^7
        xpm!(ret, i)
        y += ret[2]
    end
    return y
end;
g_inplace
@time g_inplace()

In [ ]:
# Or use an SVector/MVector, which are small (length < ~100) statically sized arrays
using StaticArrays
function xpm_static(x)
    return @SVector[x-1, x, x+1]
end

function g_static()
    y = 0
    for i = 1:10^7
        ret = xpm_static(i)
        y += ret[2]
    end
    return y
end
g_static()
@time g_static();

## Type Instability

In [ ]:
function mymax(low, x)
    s = low
    for xi in x
        s = (s < xi) ? xi : s
    end
    return s
end;

In [ ]:
# works well for integers
x = rand(Int, 1000)

@btime mymax(-Inf, $x)
@btime mymax(typemin(Int), $x)

## Slicing arrays

In [ ]:
# Let's make a Fourier series summer

c_coeffs = rand(1000) # c[m] * cos(m * θ)
s_coeffs = rand(1000) # s[m] * sin(m * θ)
c0 = 1.0

coeffs = vcat(c0, c_coeffs, s_coeffs);


In [ ]:
function Fsum(θ, coeffs)
    M = (length(coeffs) - 1) ÷ 2
    c_coeffs = coeffs[2:(M+1)]
    s_coeffs = coeffs[(M+2):end]

    tot = coeffs[1]
    tot += sum(c_coeffs .* cos.((1:M) .* θ) .+ s_coeffs .* sin.((1:M) .* θ))
end
@btime Fsum(1.0, $coeffs);

In [ ]:
θs = range(0, 2π, 1001)
@time Fsum.(θs, Ref(coeffs));

In [ ]:
function Fsum(θ, coeffs)
    M = (length(coeffs) - 1) ÷ 2
    c_coeffs = @view(coeffs[2:(M+1)])
    @views s_coeffs = coeffs[(M+2):end]

    tot = coeffs[1]
    tot += sum(c_coeffs .* cos.((1:M) .* θ) .+ s_coeffs .* sin.((1:M) .* θ))
end;
@btime Fsum(1.0, $coeffs);

In [ ]:
function Fsum(θ, coeffs)
    M = (length(coeffs) - 1) ÷ 2
    c_coeffs = @view(coeffs[2:(M+1)])
    @views s_coeffs = coeffs[(M+2):end]
    tot = coeffs[1]
    tot += sum(c_coeffs[m] * cos(m * θ) + s_coeffs[m] * sin(m * θ) for m in 1:M)
    #tot += sum(s * sin(m * θ) for (m, s) in enumerate(s_coeffs))
    return tot
end;
@btime Fsum(1.0, $coeffs);